<a href="https://colab.research.google.com/github/zwimpee/cursivetransformer/blob/main/HookedCursiveTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HookedCursiveTransformer

# Setup

In [1]:
!pip install transformer_lens
!pip install gradio
!pip install wandb
!pip install einops
!pip install matplotlib
!pip install datasets

# Clone the cursivetransformer repository and install its requirements
!rm -rf cursivetransformer && git clone https://github.com/zwimpee/cursivetransformer.git
!pip install -r cursivetransformer/requirements.txt

# Login to Weights & Biases (replace 'your_api_key' with your actual API key)
import wandb
wandb.login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.6/175.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
import sys
sys.path.append('/content/cursivetransformer')  # Adjust the path if necessary

# Import cursivetransformer modules
from cursivetransformer.model import get_all_args, get_checkpoint
from cursivetransformer.data import create_datasets, offsets_to_strokes
from cursivetransformer.sample import generate, generate_n_words, plot_strokes

# Import TransformerLens modules

import dataclasses
import logging
import os
import re
from pathlib import Path
from typing import Dict, Optional, Union

from transformer_lens import HookedTransformer, ActivationCache
from transformer_lens.components import AbstractAttention, Attention, LayerNorm, Embed, PosEmbed
from transformer_lens.components.mlps.mlp import MLP
import transformer_lens.utils as utils
from transformer_lens.HookedTransformerConfig import HookedTransformerConfig


# Import other necessary libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gradio as gr
import pprint
import json
from datasets import load_dataset
from IPython.display import HTML, display
from functools import partial
import tqdm.notebook as tqdm
import matplotlib.pyplot as plt
import math
import einops
from einops import rearrange, einsum

In [3]:
# Move input to the correct device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


args = get_all_args(False)
args.sample_only = True
args.load_from_run_id = '6le6tujz'  # Replace with your actual run ID
args.wandb_entity = 'sam-greydanus'
args.dataset_name = 'bigbank'  # Replace with your dataset name
args.wandb_run_name = 'cursivetransformer_dictionary_learning'

torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)

train_dataset, test_dataset = create_datasets(args)

args.block_size = train_dataset.get_stroke_seq_length()
args.context_block_size = train_dataset.get_text_seq_length()
args.vocab_size = train_dataset.get_vocab_size()
args.context_vocab_size = train_dataset.get_char_vocab_size()

model, optimizer, scheduler, step, best_loss = get_checkpoint(args)

Trying to load dataset file from /content/cursivetransformer/data/bigbank.json.zip
Succeeded in loading the bigbank dataset; contains 1900 items.
For a dataset of 1805 examples we can generate 440811596555 combinations of 4 examples.
Generating 497000 4-word examples.
For a dataset of 95 examples we can generate 3183545 combinations of 4 examples.
Generating 3000 4-word examples.
Number of examples in the train dataset: 497000
Number of examples in the test dataset: 3000
Max token sequence length: 1000
Number of unique characters in the ascii vocabulary: 71
Ascii vocabulary:
	" enaitoshrdx.vpukbgfcymzw1lqj804I92637OTAS5N)EHR"'(BCQLMWYU,ZF!DXV?KPGJ"
Split up the dataset into 497000 training examples and 3000 test examples
Number of Transformer parameters: 368064
Model #params: 397184
Finding latest checkpoint for W&B run id 6le6tujz
  model:best_checkpoint:v70
  model:best_checkpoint:v71
  model:best_checkpoint:v72
  model:best_checkpoint:v73
  model:best_checkpoint:v74
  model:best_che

wandb:   1 of 1 files downloaded.  


In [4]:
model

Transformer(
  (transformer): ModuleDict(
    (wte): Embedding(455, 64)
    (wpe): Embedding(1000, 64)
    (wce): Embedding(72, 64)
    (wcpe): Embedding(50, 64)
    (h): ModuleList(
      (0-3): 4 x Block(
        (ln_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=64, out_features=192, bias=True)
          (c_proj): Linear(in_features=64, out_features=64, bias=True)
        )
        (ln_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (cross_attn): CrossAttention(
          (c_attn_q): Linear(in_features=64, out_features=64, bias=True)
          (c_attn_kv): Linear(in_features=64, out_features=128, bias=True)
          (c_proj): Linear(in_features=64, out_features=64, bias=True)
        )
        (ln_3): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (mlp): ModuleDict(
          (c_fc): Linear(in_features=64, out_features=256, bias=True)
          (c_proj): Linear(in_fe

# Classes

In [5]:
class HookedCursiveTransformerConfig(HookedTransformerConfig):
    def __init__(self, **kwargs):
        # Extract custom arguments
        d_model_c = kwargs.pop('d_model_c', None)
        n_ctx_c = kwargs.pop('n_ctx_c', None)
        d_vocab_c = kwargs.pop('d_vocab_c', None)
        use_cross_attention = kwargs.pop('use_cross_attention', True)

        # Now, call the superclass constructor with the remaining kwargs
        super().__init__(**kwargs)

        # Set custom attributes after superclass initialization
        self.d_model_c = d_model_c if d_model_c is not None else self.d_model
        self.n_ctx_c = n_ctx_c if n_ctx_c is not None else self.n_ctx
        self.d_vocab_c = d_vocab_c if d_vocab_c is not None else self.d_vocab
        self.use_cross_attention = use_cross_attention


class CrossAttention(AbstractAttention):
    def __init__(self, cfg):
        super().__init__(cfg)

        self.W_Q = nn.Parameter(torch.empty(cfg.n_heads, cfg.d_model, cfg.d_head))
        self.W_K = nn.Parameter(torch.empty(cfg.n_heads, cfg.d_model_c, cfg.d_head))
        self.W_V = nn.Parameter(torch.empty(cfg.n_heads, cfg.d_model_c, cfg.d_head))
        self.W_O = nn.Parameter(torch.empty(cfg.n_heads, cfg.d_head, cfg.d_model))

        self.b_Q = nn.Parameter(torch.zeros(cfg.n_heads, cfg.d_head))
        self.b_K = nn.Parameter(torch.zeros(cfg.n_heads, cfg.d_head))
        self.b_V = nn.Parameter(torch.zeros(cfg.n_heads, cfg.d_head))
        self.b_O = nn.Parameter(torch.zeros(cfg.d_model))

        # Buffers (not parameters)
        self.register_buffer("IGNORE", torch.tensor(-1e5))
        self.register_buffer("mask", torch.tril(torch.ones((cfg.n_ctx, cfg.n_ctx_c), dtype=torch.bool)).view(1, 1, cfg.n_ctx, cfg.n_ctx_c))

    def forward(self, query_input, key_input, value_input, **kwargs):
        # Compute queries
        q = einsum('batch pos d_model, n_heads d_model d_head -> batch n_heads pos d_head', query_input, self.W_Q) + self.b_Q

        # Compute keys and values from context
        k = einsum('batch pos d_model_c, n_heads d_model_c d_head -> batch n_heads pos d_head', key_input, self.W_K) + self.b_K
        v = einsum('batch pos d_model_c, n_heads d_model_c d_head -> batch n_heads pos d_head', value_input, self.W_V) + self.b_V

        # Compute attention scores
        attn_scores = einsum('batch n_heads pos_q d_head, batch n_heads pos_k d_head -> batch n_heads pos_q pos_k', q, k)
        attn_scores = attn_scores / math.sqrt(self.cfg.d_head)

        # Apply softmax
        attn_probs = F.softmax(attn_scores, dim=-1)

        # Compute output
        z = einsum('batch n_heads pos_q pos_k, batch n_heads pos_k d_head -> batch n_heads pos_q d_head', attn_probs, v)
        z = einsum('batch n_heads pos_q d_head, n_heads d_head d_model -> batch pos_q d_model', z, self.W_O) + self.b_O

        return z



class CursiveTransformerBlock(nn.Module):
    def __init__(self, cfg, layer_idx):
        super().__init__()
        self.cfg = cfg
        self.layer_idx = layer_idx

        # Layer norms
        self.ln1 = LayerNorm(cfg)
        self.ln2 = LayerNorm(cfg)
        self.ln3 = LayerNorm(cfg)

        # Self-attention using Attention
        self.attn = Attention(cfg)

        # Cross-attention using the implemented CrossAttention
        self.cross_attn = CrossAttention(cfg)

        # MLP
        self.mlp = MLP(cfg)

    def forward(self, x, c, **kwargs):
        # Self-attention
        x_ln = self.ln1(x)
        x = x + self.attn(x_ln, x_ln, x_ln, **kwargs)

        # Cross-attention
        x_ln = self.ln2(x)
        x = x + self.cross_attn(x_ln, c, c, **kwargs)

        # MLP
        x = x + self.mlp(self.ln3(x), **kwargs)

        return x

class HookedCursiveTransformer(HookedTransformer):
    def __init__(self, cfg):
        super().__init__(cfg)
        self.cfg = cfg

        # Create a modified config for context embeddings
        import copy
        cfg_c = copy.deepcopy(cfg)
        cfg_c.d_vocab = cfg.d_vocab_c
        cfg_c.d_model = cfg.d_model_c
        cfg_c.n_ctx = cfg.n_ctx_c

        # Use transformer_lens components with the modified config
        self.embed_c = Embed(cfg_c)
        self.pos_embed_c = PosEmbed(cfg_c)

        # Override the transformer blocks with custom blocks that include cross-attention
        self.blocks = nn.ModuleList([
            CursiveTransformerBlock(self.cfg, layer_idx)
            for layer_idx in range(self.cfg.n_layers)
        ])

        # Use transformer_lens's LayerNorm
        self.ln_final = LayerNorm(cfg)

    def forward(self, x, c, return_type="logits", **kwargs):
        # Embedding and positional encoding for x (main sequence)
        x_tokens = self.embed(x)  # [batch, seq_len, d_model]
        x_positions = self.pos_embed(torch.arange(x_tokens.size(1), device=x_tokens.device))  # [seq_len, d_model]
        x = x_tokens + x_positions

        # Embedding and positional encoding for c (context)
        c_tokens = self.embed_c(c)  # [batch, context_len, d_model_c]
        c_positions = self.pos_embed_c(torch.arange(c_tokens.size(1), device=c_tokens.device))  # [context_len, d_model_c]
        c = c_tokens + c_positions

        # Pass through transformer blocks
        for block in self.blocks:
            x = block(x, c, **kwargs)

        x = self.ln_final(x)
        logits = self.unembed(x)

        # Handle return types as before
        if return_type == "logits":
            return logits
        elif return_type == "loss":
            targets = kwargs.get('targets')
            if targets is None:
                raise ValueError("Targets must be provided when return_type is 'loss'")
            loss = nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
            return loss
        elif return_type == "both":
            return logits, kwargs.get('cache', None)
        else:
            raise ValueError(f"Invalid return_type: {return_type}")


# Functions

In [6]:
def convert_cursivetransformer_model_config(args):
    cfg_dict = {
        # Standard parameters
        "d_model": args.n_embd,
        "n_layers": args.n_layer,
        "d_mlp": args.n_embd * 4,
        "d_head": args.n_embd // args.n_ctx_head,
        "n_heads": args.n_ctx_head,
        "n_ctx": args.max_seq_length,
        "d_vocab": args.vocab_size,
        "tokenizer_name": None,
        "act_fn": "gelu_new",
        "attn_only": False,
        "final_rms": False,
        "original_architecture": "cursivetransformer",
        "normalization_type": "LN",
        "init_weights": False,
        "device": args.device,
        # Additional parameters for cross-attention
        "d_model_c": args.n_embd2,
        "n_ctx_c": args.context_block_size,
        "d_vocab_c": args.context_vocab_size,
        "use_cross_attention": True,
    }
    cfg = HookedCursiveTransformerConfig.from_dict(cfg_dict)
    return cfg

# - [ ] TODO: Determine if this is actually needed for us, I suspect it might be vestigial
def fill_missing_keys(model, state_dict):
    """Takes in a state dict from a pretrained model, and fills in any missing keys with the default initialization.

    This function is assumed to be run before weights are initialized.

    Args:
        state_dict (dict): State dict from a pretrained model

    Returns:
        dict: State dict with missing keys filled in
    """
    # Get the default state dict
    default_state_dict = model.state_dict()
    # Get the keys that are missing from the pretrained model
    missing_keys = set(default_state_dict.keys()) - set(state_dict.keys())
    # Fill in the missing keys with the default initialization
    for key in missing_keys:
        if "hf_model" in key:
            # Skip keys that are from the HuggingFace model, if loading from HF.
            continue
        if "W_" in key:
            logging.warning(
                "Missing key for a weight matrix in pretrained, filled in with an empty tensor: {}".format(
                    key
                )
            )
        state_dict[key] = default_state_dict[key]
    return state_dict

def convert_cursivetransformer_weights(cursivetransformer, cfg):
    state_dict = {}
    # Embeddings
    state_dict["embed.W_E"] = cursivetransformer.transformer.wte.weight
    state_dict["pos_embed.W_pos"] = cursivetransformer.transformer.wpe.weight
    state_dict["embed_c.W_E"] = cursivetransformer.transformer.wce.weight
    state_dict["pos_embed_c.W_pos"] = cursivetransformer.transformer.wcpe.weight

    for l in range(cfg.n_layers):
        block = cursivetransformer.transformer.h[l]

        # Layer Norms (adjusted parameter names)
        state_dict[f'blocks.{l}.ln1.w'] = block.ln_1.weight
        state_dict[f'blocks.{l}.ln1.b'] = block.ln_1.bias
        state_dict[f'blocks.{l}.ln2.w'] = block.ln_2.weight
        state_dict[f'blocks.{l}.ln2.b'] = block.ln_2.bias
        state_dict[f'blocks.{l}.ln3.w'] = block.ln_3.weight
        state_dict[f'blocks.{l}.ln3.b'] = block.ln_3.bias

        # Self-Attention Weights
        self_attn = block.attn
        W_self = self_attn.c_attn.weight  # Shape: [3 * d_model, d_model]
        b_self = self_attn.c_attn.bias    # Shape: [3 * d_model]

        # Split weights and biases
        W_Q_self, W_K_self, W_V_self = torch.chunk(W_self, 3, dim=0)
        b_Q_self, b_K_self, b_V_self = torch.chunk(b_self, 3, dim=0)

        # Reshape and assign
        W_Q_self = W_Q_self.T.reshape(cfg.n_heads, cfg.d_model, cfg.d_head)
        W_K_self = W_K_self.T.reshape(cfg.n_heads, cfg.d_model, cfg.d_head)
        W_V_self = W_V_self.T.reshape(cfg.n_heads, cfg.d_model, cfg.d_head)
        state_dict[f'blocks.{l}.attn.W_Q'] = W_Q_self
        state_dict[f'blocks.{l}.attn.W_K'] = W_K_self
        state_dict[f'blocks.{l}.attn.W_V'] = W_V_self
        state_dict[f'blocks.{l}.attn.b_Q'] = b_Q_self.reshape(cfg.n_heads, cfg.d_head)
        state_dict[f'blocks.{l}.attn.b_K'] = b_K_self.reshape(cfg.n_heads, cfg.d_head)
        state_dict[f'blocks.{l}.attn.b_V'] = b_V_self.reshape(cfg.n_heads, cfg.d_head)

        # Self-Attention Output Projection
        W_O_self = self_attn.c_proj.weight  # Shape: [d_model, d_model]
        b_O_self = self_attn.c_proj.bias    # Shape: [d_model]
        W_O_self = W_O_self.T.reshape(cfg.n_heads, cfg.d_head, cfg.d_model)
        state_dict[f'blocks.{l}.attn.W_O'] = W_O_self
        state_dict[f'blocks.{l}.attn.b_O'] = b_O_self

        # Cross-Attention Weights
        cross_attn = block.cross_attn
        W_Q_cross = cross_attn.c_attn_q.weight  # Shape: [d_model, d_model]
        b_Q_cross = cross_attn.c_attn_q.bias    # Shape: [d_model]
        W_KV_cross = cross_attn.c_attn_kv.weight  # Shape: [2 * d_model_c, d_model_c]
        b_KV_cross = cross_attn.c_attn_kv.bias    # Shape: [2 * d_model_c]

        # Split KV weights and biases
        W_K_cross, W_V_cross = torch.chunk(W_KV_cross, 2, dim=0)
        b_K_cross, b_V_cross = torch.chunk(b_KV_cross, 2, dim=0)

        # Reshape and assign
        W_Q_cross = W_Q_cross.T.reshape(cfg.n_heads, cfg.d_model, cfg.d_head)
        W_K_cross = W_K_cross.T.reshape(cfg.n_heads, cfg.d_model_c, cfg.d_head)
        W_V_cross = W_V_cross.T.reshape(cfg.n_heads, cfg.d_model_c, cfg.d_head)
        state_dict[f'blocks.{l}.cross_attn.W_Q'] = W_Q_cross
        state_dict[f'blocks.{l}.cross_attn.W_K'] = W_K_cross
        state_dict[f'blocks.{l}.cross_attn.W_V'] = W_V_cross
        state_dict[f'blocks.{l}.cross_attn.b_Q'] = b_Q_cross.reshape(cfg.n_heads, cfg.d_head)
        state_dict[f'blocks.{l}.cross_attn.b_K'] = b_K_cross.reshape(cfg.n_heads, cfg.d_head)
        state_dict[f'blocks.{l}.cross_attn.b_V'] = b_V_cross.reshape(cfg.n_heads, cfg.d_head)

        # Cross-Attention Output Projection
        W_O_cross = cross_attn.c_proj.weight  # Shape: [d_model, d_model]
        b_O_cross = cross_attn.c_proj.bias    # Shape: [d_model]
        W_O_cross = W_O_cross.T.reshape(cfg.n_heads, cfg.d_head, cfg.d_model)
        state_dict[f'blocks.{l}.cross_attn.W_O'] = W_O_cross
        state_dict[f'blocks.{l}.cross_attn.b_O'] = b_O_cross

        # MLP Weights
        mlp = block.mlp
        W_in = mlp.c_fc.weight  # Shape: [4 * d_model, d_model]
        b_in = mlp.c_fc.bias    # Shape: [4 * d_model]
        W_out = mlp.c_proj.weight  # Shape: [d_model, 4 * d_model]
        b_out = mlp.c_proj.bias    # Shape: [d_model]

        # Transpose and assign
        state_dict[f'blocks.{l}.mlp.W_in'] = W_in.T
        state_dict[f'blocks.{l}.mlp.b_in'] = b_in
        state_dict[f'blocks.{l}.mlp.W_out'] = W_out.T
        state_dict[f'blocks.{l}.mlp.b_out'] = b_out

    # Unembedding Weights
    state_dict["unembed.W_U"] = cursivetransformer.lm_head.weight.T
    if cursivetransformer.lm_head.bias is not None:
        state_dict["unembed.b_U"] = cursivetransformer.lm_head.bias
    else:
        state_dict["unembed.b_U"] = torch.zeros(cfg.d_vocab_out)

    # Final Layer Norm (adjusted parameter names)
    state_dict["ln_final.w"] = cursivetransformer.transformer.ln_f.weight
    state_dict["ln_final.b"] = cursivetransformer.transformer.ln_f.bias

    return state_dict

# Sandbox

In [7]:
# Ensure the configuration has the correct maximum sequence lengths
cfg = convert_cursivetransformer_model_config(args)
state_dict = convert_cursivetransformer_weights(model, cfg)
hooked_model = HookedCursiveTransformer(cfg)
hooked_model.load_state_dict(state_dict, strict=False)
hooked_model.to(device)

# Add batch dimension to your inputs
x, c, y = test_dataset[0]
x = x.unsqueeze(0).to(device)  # Shape: [1, 1000]
c = c.unsqueeze(0).to(device)  # Shape: [1, 50]
y = y.unsqueeze(0).to(device) # Shape: [1, 1000]

print(x.shape)  # torch.Size([1, 1000])
print(c.shape)  # torch.Size([1, 50])
print(y.shape)  # torch.Size([1, 1000])

# Run the model with cache
hooked_model.eval()
with torch.no_grad():
    logits, cache = hooked_model(x, c, return_type="both")

# Choose the layer and head to visualize
layer = 0  # Change to the desired layer index
head = 0   # Change to the desired head index

# Access attention patterns for the specified layer
attn_patterns = cache[f'blocks.{layer}.attn.hook_pattern']  # Shape: [batch_size, n_heads, seq_len, seq_len]

# Extract the attention pattern for the specified head and sample
attn = attn_patterns[0, head].cpu().numpy()  # Shape: [seq_len, seq_len]

plt.figure(figsize=(8, 6))
plt.imshow(attn, cmap='viridis', aspect='auto')
plt.colorbar()
plt.title(f'Self-Attention Pattern for Layer {layer}, Head {head}')
plt.xlabel('Key Positions')
plt.ylabel('Query Positions')
plt.show()

Moving model to device:  cuda
torch.Size([1, 1000])
torch.Size([1, 50])
torch.Size([1, 1000])


OutOfMemoryError: CUDA out of memory. Tried to allocate 14.90 GiB. GPU 0 has a total capacity of 39.56 GiB of which 4.04 GiB is free. Process 6501 has 35.51 GiB memory in use. Of the allocated memory 34.99 GiB is allocated by PyTorch, and 31.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Limit cache to specific layer and head
layer = 0  # Layer you're interested in
head = 0   # Head you're interested in

def cache_filter(name):
    return name == f'blocks.{layer}.attn.hook_pattern'

with torch.no_grad(), torch.cuda.amp.autocast():
    logits, cache = hooked_model.run_with_cache(
        x, c,
        return_type="logits",
        names_filter=cache_filter
    )

# Proceed to extract and visualize the attention patterns
attn_patterns = cache[f'blocks.{layer}.attn.hook_pattern']  # [batch_size, n_heads, seq_len_q, seq_len_k]
attn = attn_patterns[0, head].cpu().numpy()  # [seq_len_q, seq_len_k]

# Visualization code remains the same
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.imshow(attn, cmap='viridis', aspect='auto')
plt.colorbar()
plt.title(f'Self-Attention Pattern for Layer {layer}, Head {head}')
plt.xlabel('Key Positions')
plt.ylabel('Query Positions')
plt.show()
